In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
data = pd.read_csv("https://raw.githubusercontent.com/jbrownlee/Datasets/master/wheat-seeds.csv")

In [3]:
# 6 Step
# Initialize Network
# Forward Propagete.
# Back Propagate Error,
# Train Network.
# Predict
# Seed Datasets Case Study


# These steps will provide the foundation that you need to implement the backpropagation algorithm 
# from scratch and apply it to your own predictive modeling problems.



In [4]:
# 1. Initialize Network
# Each neuron has a set of weights that need to be maintained
# One weight for each input connection and an additional weight for the bias.
# A network is organized into layers.
# The first real layer is the hidden layer. 
# This is followed by the output layer that has one neuron for each class value.

In [5]:
# initialize_network() that creates a new neural network ready for training.
# It accepts three parameters, the number of inputs, 
# the number of neurons to have in the hidden layer and the number of outputs.

# n_hidden neurons and each neuron in the hidden layer has n_inputs + 1 weights, 
# one for each input column in a dataset and an additional one for the bias.

# the output layer that connects to the hidden layer has n_outputs neurons, each with n_hidden + 1 weights.
# each neuron in the output layer connects to (has a weight for) each neuron in the hidden layer.

In [7]:

from random import seed
from random import random
 
# Initialize a network
def initialize_network(n_inputs, n_hidden, n_outputs):
    network = list()
    hidden_layer = [{'weights':[random() for i in range(n_inputs + 1)]} for i in range(n_hidden)]
    network.append(hidden_layer)
    output_layer = [{'weights':[random() for i in range(n_hidden + 1)]} for i in range(n_outputs)]
    network.append(output_layer)
    return network
 
seed(1)
network = initialize_network(2, 1, 2)
for layer in network:
    print(layer)

[{'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614]}]
[{'weights': [0.2550690257394217, 0.49543508709194095]}, {'weights': [0.4494910647887381, 0.651592972722763]}]


In [19]:
# Forward Propagate
# We can calculate an output from a neural network by propagating an input signal through
# each layer until the output layer outputs it's values
# Calculate neuron activation for an input
# activate mean linear regresion: y(activation) = ax +b depends on the dataset you have
def activate(weights, inputs):
    activation = weights[-1]
    for i in range(len(weights) - 1):
        activation += weights[i] * inputs[i]
    return activation
# like sigmoid, which probability each outcome have 
# 1/(1+ e^ activation)
def transfer(activation):
    return 1.0/(1.0+ np.exp(-activation))
def forward_propagate(network, row):
    inputs = row
    for layer in network:
        new_inputs = []
        for neuron in layer:
            activation = activate(neuron['weights'], inputs)
            neuron['output'] = transfer(activation)
            new_inputs.append(neuron['output'])
        inputs = new_inputs
    return inputs
network = [[{'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614]}],
           [{'weights': [0.2550690257394217, 0.49543508709194095]}, {'weights': [0.4494910647887381, 0.651592972722763]}]]
row = [1, 0, None]
output = forward_propagate(network, row)
print(output)

[0.6629970129852887, 0.7253160725279748]


In [22]:
# https://machinelearningmastery.com/implement-backpropagation-algorithm-scratch-python/
# Back Propagate Error
# The backpropagation algorithm is named for the way in which weights are trained.
# Error is calculated between the expected outputs and the outputs forward propagated from the network
# These errors are then propagated backward through the network from the output layer to the hidden layer, 
# assigning blame for the error and updating weights as they go.
# This part is broken down into two sections.
    # Transfer Derivative.
    # Error Backpropagation.

In [20]:
# 3.1. Transfer Derivative
# Given an output value from a neuron, we need to calculate it’s slope.
def transfer_derivative(output):
    return output * (1.0 - output)

In [28]:
# 3.2. Error Backpropagation

def backward_propagate_error(network, expected):
    for i in reversed(range(len(network))):
        layer = network[i]
        errors = list()
        if i != len(network)-1:
            for j in range(len(layer)):
                error = 0.0
                for neuron in network[i + 1]:
                    error += (neuron['weights'][j] * neuron['delta'])
                errors.append(error)
        else:
            for j in range(len(layer)):
                neuron = layer[j]
                errors.append(expected[j] - neuron['output'])
        for j in range(len(layer)):
            neuron = layer[j]
            neuron['delta'] = errors[j] * transfer_derivative(neuron['output'])
network = [[{'output': 0.7105668883115941, 'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614]}],
[{'output': 0.6213859615555266, 'weights': [0.2550690257394217, 0.49543508709194095]}, {'output': 0.6573693455986976, 'weights': [0.4494910647887381, 0.651592972722763]}]]
expected = [0, 1]
backward_propagate_error(network, expected)
for layer in network:
    print(layer)

[{'output': 0.7105668883115941, 'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614], 'delta': -0.0005348048046610517}]
[{'output': 0.6213859615555266, 'weights': [0.2550690257394217, 0.49543508709194095], 'delta': -0.14619064683582808}, {'output': 0.6573693455986976, 'weights': [0.4494910647887381, 0.651592972722763], 'delta': 0.0771723774346327}]


In [30]:
# The network is trained using stochastic gradient descent.
# backpropagating the error and updating the network weights.
def update_weights(network, row, l_rate):
    for i in range(len(network)):
        inputs = row[:-1]
        if i != 0:
            inputs = [neuron['output'] for neuron in network[i - 1]]
        for neuron in network[i]:
            for j in range(len(inputs)):
                neuron['weights'][j] += l_rate * neuron['delta'] * inputs[j]
            neuron['weights'][-1] += l_rate * neuron['delta']

In [41]:
def train_network(network, train, l_rate, n_epoch, n_outputs):
    for epoch in range(n_epoch):
        sum_error = 0
        for row in train:
            outputs = forward_propagate(network, row)
            expected = [0 for i in range(n_outputs)]
            expected[row[-1]] = 1
            sum_error += sum([(expected[i]-outputs[i])**2 for i in range(len(expected))])
            backward_propagate_error(network, expected)
            update_weights(network, row, l_rate)
        print('>epoch=%d, lrate=%.3f, error=%.3f' % (epoch, l_rate, sum_error))
def predict(network, row):
    outputs = forward_propagate(network, row)
    return outputs.index(max(outputs))

In [45]:
dataset = [[2.7810836,2.550537003,0],
	[1.465489372,2.362125076,0],
	[3.396561688,4.400293529,0],
	[1.38807019,1.850220317,0],
	[3.06407232,3.005305973,0],
	[7.627531214,2.759262235,1],
	[5.332441248,2.088626775,1],
	[6.922596716,1.77106367,1],
	[8.675418651,-0.242068655,1],
	[7.673756466,3.508563011,1]]

n_inputs = len(dataset[0]) - 1
# how many output categorical
n_outputs = len(set([row[-1] for row in dataset]))
n_outputs
network = initialize_network(n_inputs, 2, n_outputs)
network
train_network(network, dataset, 0.5, 20, n_outputs)
for layer in network:
	print(layer)

>epoch=0, lrate=0.500, error=6.819
>epoch=1, lrate=0.500, error=5.917
>epoch=2, lrate=0.500, error=5.372
>epoch=3, lrate=0.500, error=5.251
>epoch=4, lrate=0.500, error=5.240
>epoch=5, lrate=0.500, error=5.213
>epoch=6, lrate=0.500, error=5.148
>epoch=7, lrate=0.500, error=5.038
>epoch=8, lrate=0.500, error=4.885
>epoch=9, lrate=0.500, error=4.695
>epoch=10, lrate=0.500, error=4.468
>epoch=11, lrate=0.500, error=4.207
>epoch=12, lrate=0.500, error=3.921
>epoch=13, lrate=0.500, error=3.622
>epoch=14, lrate=0.500, error=3.324
>epoch=15, lrate=0.500, error=3.035
>epoch=16, lrate=0.500, error=2.764
>epoch=17, lrate=0.500, error=2.515
>epoch=18, lrate=0.500, error=2.289
>epoch=19, lrate=0.500, error=2.086
[{'weights': [1.2508178628624307, -1.70839446417483, -0.3991177297838095], 'output': 0.9446393078263786, 'delta': 0.010205776833704739}, {'weights': [0.8443968750118955, 0.4928715923394345, 0.2093969687445748], 'output': 0.9997793238185477, 'delta': -6.357268489583483e-06}]
[{'weights': [-

In [46]:
# https://machinelearningmastery.com/implement-backpropagation-algorithm-scratch-python/
for row in dataset:
	prediction = predict(network, row)
	print('Expected=%d, Got=%d' % (row[-1], prediction))

Expected=0, Got=0
Expected=0, Got=0
Expected=0, Got=0
Expected=0, Got=0
Expected=0, Got=0
Expected=1, Got=1
Expected=1, Got=1
Expected=1, Got=1
Expected=1, Got=1
Expected=1, Got=1


In [ ]:
# What Is an Epoch?

# The number of epochs is a hyperparameter that defines the number times that 
# the learning algorithm will work through the entire training dataset.

# One epoch means that each sample in the training dataset has had an opportunity 
# to update the internal model parameters.


In [ ]:
# Back Propagation and Weight Updation like update weight to make the output must to minimal errors.
